### Problem 1

#### Imports

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

#### Inputs
I added a "cashflows" variable cf, which is non-zero only for the initial period. This setup can be used to model additional flows coming in/out of the portfolio throughout the investment period

In [2]:
var_len = 13
cf = np.zeros(var_len)
cf[0] = 10
r_equity = 0.06
r_second_pe = 0.12
r_prim_pe = 0.16

#### Define Variables

In [3]:
# equities
x = cp.Variable(shape=var_len, nonneg=True, name='equity')
# secondary PE
y = cp.Variable(shape=var_len, nonneg=True, name='pe_secondary')
# primary PE
z = cp.Variable(shape=var_len, nonneg=True, name='pe_primary')
# wealth
w = cp.Variable(shape=var_len, nonneg=True, name='wealth')

#### Create Constraints
This creates 2 constraints for each period in the analysis
- budgeting constraint, which determines how much capital can be deployed to the investment vehicles
- liquidity constraint, which drives the maximum avg remaining holding period

In [4]:
constraints = list()
constraint_values = dict()

for t in range(0, var_len):
    # new investments
    con_wealth_t = - x[t] - y[t] - z[t]

    con_liq_t = -x[t] + y[t] + 3*z[t]
    
    # add existing equity value
    if t - 1 >= 0:
        con_wealth_t = con_wealth_t + (1+r_equity) * x[t-1]
        con_liq_t = con_liq_t - (1+r_equity) * x[t-1]
    # add secondary PE
    if t - 3 >= 0:
        con_wealth_t = con_wealth_t + (1+r_second_pe)**3 * y[t-3]
        con_liq_t = con_liq_t + (1+r_second_pe)**3 * y[t-3]
    # add primary PE
    if t - 5 >= 0:
        con_wealth_t = con_wealth_t + (1+r_prim_pe)**5 * z[t-5]
        con_liq_t = con_liq_t + 3 * (1+r_prim_pe)**5 * z[t-5]

    # value of today's investments (neg sign!) plus previous period's investments is equal to total wealth
    constraint_values[f'budget_{t}'] = {
        'lhs': con_wealth_t + cf[t],
        'rhs': w[t],
    }
    constraints.append(con_wealth_t + cf[t] == w[t])

    # liquidity constraint
    constraint_values[f'liquidity_{t}'] = {
        'lhs': con_liq_t,
        'rhs': 0,
    }    
    constraints.append(con_liq_t <= 0)

#### Objective
Maximixing the final wealth. This also takes care of the question of all assets being liquidated by end of year 12

In [5]:
obj = cp.Maximize(w[-1])

#### Problem setup and solve

In [6]:
prob = cp.Problem(objective=obj, constraints=constraints)

In [7]:
prob.solve(verbose=False)

np.float64(37.64337052935727)

#### Values of all variables

In [8]:
df = pd.DataFrame({
    'equity': x.value,
    'secondary pe': y.value,
    'primary pe': z.value,
    'wealth': w.value
}).round(2)
df.index.name = 'period'
df

,equity,secondary pe,primary pe,wealth
period,,,,
0,7.04,0.92,2.04,0.00
1,1.32,4.83,1.32,0.00
2,0.81,0.00,0.58,0.00
3,1.72,0.00,0.43,0.00
4,7.69,0.00,0.91,0.00
5,10.51,0.00,1.94,0.00
6,9.72,0.00,4.18,0.00
7,6.94,0.11,4.48,0.00
8,1.81,6.45,0.00,0.00


#### Constraint Review
Check which are active and which are inactive

In [9]:
active_cons = list()
inactive_cons = list()
for name, value in constraint_values.items():
    rhs = value['rhs']
    if isinstance(rhs, cp.Expression):
        rhs = rhs.value
    if value['lhs'].value <= rhs:
        active_cons.append(name)
    else:
        inactive_cons.append(name)

#### Active constraints
The naming convention is either by budget or liquidity, and the index signifies at the end of which period the constraint is defined.

In [10]:
active_cons

['budget_0',
 'liquidity_0',
 'budget_1',
 'liquidity_1',
 'budget_2',
 'liquidity_2',
 'liquidity_3',
 'budget_4',
 'liquidity_4',
 'liquidity_5',
 'liquidity_6',
 'liquidity_7',
 'liquidity_8',
 'budget_9',
 'liquidity_9',
 'budget_10',
 'liquidity_10',
 'liquidity_11',
 'budget_12',
 'liquidity_12']

#### Inactive constraints
The naming convention is either by budget or liquidity, and the index signifies at the end of which period the constraint is defined.

In [11]:
inactive_cons

['budget_3', 'budget_5', 'budget_6', 'budget_7', 'budget_8', 'budget_11']